In [1]:
import pandas as pd
from src.utils.data_utils import get_demand_df, update_demand_dic, load_matrix_df, get_demand_matrix_df, load_df
from src.utils.helper_utils import get_penalty_list, get_values_not_in_second_list
from src.utils.route_utils import sort_nodes_by_distance

In [2]:
demand_df = get_demand_df(
    today_path='../data/orders/03-03-2025-PO.csv'
    )
demand_df.shape

(457, 10)

In [3]:
demand_dict = update_demand_dic(demand_df)

In [4]:
distance_mat = load_matrix_df('../data/master/osrm_distance_matrix.csv')

In [5]:
demand_mat_df = get_demand_matrix_df(distance_mat,demand_df, 0)

In [6]:
today = '2025-03-03'
DISTANCE_BASE_PENALTY = 1000
penalty_lis = get_penalty_list(demand_dict, DISTANCE_BASE_PENALTY, 6, today)

In [7]:
demand_sort_nodes = sort_nodes_by_distance(demand_mat_df)

In [8]:
nodes_to_visit = demand_sort_nodes[:300]

In [9]:
# from src.data_model.vrp_data_model import VRPDataModel
"""
Data model for the Vehicle Routing Problem.
Creates the data model for the solver based on the input data.
"""

import src.config as config

class VRPDataModel:
    def __init__(self, full_matrix, nodes_to_visit, demand_dict,
                 penalty_list=None, max_distance=None, max_visits=None, depot_code='0'):
        
        if max_distance is None:
            max_distance = config.MAX_DISTANCE_PER_VEHICLE
        if max_visits is None:
            max_visits = config.MAX_VISITS_PER_VEHICLE

        self.full_matrix = full_matrix  # Pandas DataFrame with named rows/columns
        self.nodes_to_visit = nodes_to_visit  # e.g., [1005, 1017, ...]
        self.demand_dict = demand_dict  # Dict: {shop_code: demand}
        self.penalty_list = penalty_list
        self.max_distance = max_distance
        self.max_visits = max_visits
        self.depot_code = depot_code  # Custom depot code, e.g., 'SMAK'

        self.data = {}
        self._create_model()

    def _create_model(self):
        print("create_data_model called with:")
        print(f"Depot: {self.depot_code}")
        print(f"Max Distance: {self.max_distance}")
        print(f"Max Visits: {self.max_visits}")
        print(f"Distance Matrix Shape: {self.full_matrix.shape}")
        print(f"Nodes to Visit: {len(self.nodes_to_visit)}")

        # Validate that depot exists
        valid_keys = set(self.full_matrix.index).intersection(self.full_matrix.columns)
        assert self.depot_code in valid_keys, f"Depot '{self.depot_code}' not found in distance matrix."

        # Filter out invalid nodes and exclude depot from visit list
        filtered_nodes = [node for node in self.nodes_to_visit if node in valid_keys and node != self.depot_code]

        # Build node list (depot always at index 0)
        all_nodes = [self.depot_code] + filtered_nodes

        # Build distance matrix
        self.data["distance_matrix"] = [
            [self.full_matrix.loc[from_node, to_node] for to_node in all_nodes]
            for from_node in all_nodes
        ]

        # Build demand list
        self.data["demands"] = [0] + [self.demand_dict.get(node, 1) for node in filtered_nodes]

        # Set other model fields
        self.data["node_mapping"] = all_nodes
        self.data["num_vehicles"] = len(self.max_distance)
        self.data["depot"] = 0
        self.data["max_distance_per_vehicle"] = self.max_distance
        self.data["max_visits_per_vehicle"] = self.max_visits

        # Build penalty list (optional)
        if self.penalty_list is not None:
            filtered_penalties = [
                penalty for node, penalty in zip(self.nodes_to_visit, self.penalty_list)
                if node in filtered_nodes
            ]
            self.data["penalties"] = [0] + filtered_penalties
        else:
            self.data["penalties"] = [0] + [1000] * len(filtered_nodes)

    def get_data(self):
        return self.data


In [17]:
dataClass = VRPDataModel(
    full_matrix=demand_mat_df,
    nodes_to_visit=nodes_to_visit,
    demand_dict=demand_dict,
    penalty_list=penalty_lis,
    max_visits=[8]*8,
    depot_code='1108'
)

create_data_model called with:
Depot: 1108
Max Distance: [200, 200, 200, 200, 200, 200, 200, 200]
Max Visits: [8, 8, 8, 8, 8, 8, 8, 8]
Distance Matrix Shape: (450, 450)
Nodes to Visit: 300


In [18]:
data=dataClass.get_data()

In [19]:
len(data['distance_matrix'])

300

In [20]:
from src.solver.vrp_solver import VRPSolver

In [21]:
solver = VRPSolver()

In [22]:
visited_nodes, route_dict = solver.solve_day(data=data, day=1)


Day 2 Routes:
Route for vehicle 0:
 1108 -> 1872 -> 1471 -> 1320 -> SCCR -> SCRJ -> SCNW -> SCSJ -> 1614 -> 1108
Distance of route: 15 km (Max: 200)
Stops visited: 8/8

Route for vehicle 1:
 1108 -> 1005 -> 1411 -> 1130 -> 1308 -> 1033 -> SCMD -> 1594 -> 1291 -> 1108
Distance of route: 15 km (Max: 200)
Stops visited: 8/8

Route for vehicle 2:
 1108 -> 1364 -> 14003 -> 1303 -> 1676 -> 14011 -> 1761 -> 1859 -> 1446 -> 1108
Distance of route: 11 km (Max: 200)
Stops visited: 8/8

Route for vehicle 3:
 1108 -> 1145 -> 1323 -> 1107 -> SCTB -> 1419 -> SCDP -> SCTU -> 1433 -> 1108
Distance of route: 13 km (Max: 200)
Stops visited: 8/8

Route for vehicle 4:
 1108 -> SCMY -> 1137 -> SGKR -> 3 -> SGMH -> 14015 -> SGWA -> 1555 -> 1108
Distance of route: 21 km (Max: 200)
Stops visited: 8/8

Route for vehicle 5:
 1108 -> 1292 -> 1896 -> 1746 -> 1301 -> 1776 -> 1497 -> 1728 -> SCK7 -> 1108
Distance of route: 7 km (Max: 200)
Stops visited: 8/8

Route for vehicle 6:
 1108 -> 1545 -> 1759 -> SCSL -> 10

In [1]:
from src.controller.controller import VRPController

# === File paths ===
demand_path = '../data/orders/03-03-2025-PO.csv'
matrix_path = '../data/master/osrm_distance_matrix.csv'
gps_path = '../data/master/master_gps.csv'
today = '2025-03-03'
DISTANCE_BASE_PENALTY = 1000
total_days = 6

# === Initialize Controller ===
controller = VRPController()

controller.load_inputs(
    demand_path=demand_path,
    matrix_path=matrix_path,
    gps_path=gps_path,
    base_penalty=DISTANCE_BASE_PENALTY,
    total_days=total_days,
    today=today
)

# === Predefined route assignment ===
vehicle_routes = {
    0: 1,  # vehicle_id 0 uses route_id 1 from predefined routes
    1: 2
}

# === Configure Vehicles ===
controller.configure_vehicles(
    num_vehicles=8,
    max_visits=[12] * 8,
    max_distance=[400] * 8,
    vehicle_routes=vehicle_routes
)

# === Solve VRP ===
visited_nodes, routes = controller.solve_single_day()

# === Print results ===
print("Visited Nodes (Predefined + Solved):", visited_nodes)
print("Routes by Vehicle ID:")
for vid, route in routes.items():
    print(f"Vehicle {vid}: {route}")

shop codes len:32
create_data_model called with:
max_distance: [400]
max_visits: [12]
Distance Matrix Shape: (651, 651)
Length of nodes_to_visit: 32

Day 2 Routes:
Route for vehicle 0:
 0 -> 1331 -> SINL -> SIWD -> SIP2 -> 1781 -> SFAB -> 1466 -> 1117 -> 1847 -> 1370 -> 1809 -> 1787 -> 0
Distance of route: 239 km (Max: 400)
Stops visited: 12/12

Max route distance for Day 2: 239 km

{0: {'route_nodes': ['0', '1331', 'SINL', 'SIWD', 'SIP2', '1781', 'SFAB', '1466', '1117', '1847', '1370', '1809', '1787', '0'], 'route_distance': 239, 'max_distance_limit': 400, 'within_limit': True, 'num_visits': 12, 'max_visits_limit': 12}}
shop codes len:22
create_data_model called with:
max_distance: [400]
max_visits: [12]
Distance Matrix Shape: (651, 651)
Length of nodes_to_visit: 22

Day 2 Routes:
Route for vehicle 0:
 0 -> 1170 -> SKKM -> 1460 -> 1113 -> 122 -> 1159 -> SYMT -> 1659 -> 1481 -> 1124 -> 1350 -> 1770 -> 0
Distance of route: 263 km (Max: 400)
Stops visited: 12/12

Max route distance for D

In [23]:
from src.utils.visualization import visualize_routes_per_vehicle, load_route_cache, print_route_summary, save_route_details_to_csv

In [24]:
load_route_cache()

Loaded 151 cached routes from ../data/csv/route_cache.csv


In [ ]:
controller.load_inputs(
    demand_path=demand_path,
    matrix_path=matrix_path,
    gps_path=gps_path,
    base_penalty=DISTANCE_BASE_PENALTY,
    total_days=total_days,
    today=today
)
maps = visualize_routes_per_vehicle(
    master_df=controller.master_gps_df,
    route_dict=route_dict,
    day=0,
    use_distance=True  # or False if using time
)

No valid path found between 1130 and 1308, skipping.
No valid path found between 1107 and SCTB, skipping.
No valid path found between SCMY and 1137, skipping.
No valid path found between SGKR and 3, skipping.
No valid path found between 1105 and 1608, skipping.
Saved 206 cached routes to ../data/csv/route_cache.csv


In [2]:
from src.controller.controller import VRPController, RouteMapManager

# === File paths ===
demand_path = '../data/orders/03-03-2025-PO.csv'
matrix_path = '../data/master/osrm_distance_matrix.csv'
gps_path = '../data/master/master_gps.csv'
today = '2025-03-03'
DISTANCE_BASE_PENALTY = 1000
total_days = 6

# Setup
controller = VRPController()
controller.load_inputs(demand_path, matrix_path, gps_path, DISTANCE_BASE_PENALTY, total_days, today,depot_code='1545')
controller.configure_vehicles(8, [12]*8, [200]*8)

# Solve
visited_nodes, route_dict = controller.solve_single_day()

# Visualize & Save
visualizer = RouteMapManager(controller.master_gps_df, controller.demand_df)
visualizer.generate_and_save_maps(route_dict, day=0)
visualizer.summarize_and_save(route_dict, day=0)

create_data_model called with:
Depot: 1545
Max Distance: [200, 200, 200, 200, 200, 200, 200, 200]
Max Visits: [12, 12, 12, 12, 12, 12, 12, 12]
Distance Matrix Shape: (651, 651)
Nodes to Visit: 457

Day 2 Routes:
Route for vehicle 0:
 1545 -> SCKL -> 112 -> SCPY -> 14001 -> SCGS -> SCNG -> SCPO -> SCMU -> 172 -> 1745 -> SCGT -> SCWP -> 1545
Distance of route: 29 km (Max: 200)
Stops visited: 12/12

Route for vehicle 1:
 1545 -> SCET -> SGRD -> SCDE -> 1774 -> 1106 -> 1484 -> 23 -> SCML -> 1650 -> 25 -> 1308 -> 1130 -> 1545
Distance of route: 21 km (Max: 200)
Stops visited: 12/12

Route for vehicle 2:
 1545 -> SCMY -> 1137 -> SGWA -> 14015 -> SGKR -> 3 -> SGKG -> SGKB -> 14 -> SGTY -> SGSM -> SGKY -> 1545
Distance of route: 27 km (Max: 200)
Stops visited: 12/12

Route for vehicle 3:
 1545 -> 1872 -> 1320 -> 1471 -> SCCR -> SCSJ -> SCRJ -> SCNW -> 14008 -> SCTB -> 1419 -> 14011 -> 1676 -> 1545
Distance of route: 15 km (Max: 200)
Stops visited: 12/12

Route for vehicle 4:
 1545 -> 1005 -> 1